In [1]:
import pandas as pd
pd.options.mode.copy_on_write = True
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import os

import re
import time

import itertools

import COMBINE_harmonizer
from COMBINE_harmonizer import cfg

## 01. init

In [2]:
root_dir = '../'

In [3]:
COMBINE_harmonizer.init(f'{root_dir}/config.yaml')

out_dir = f"{cfg.config['out_dir']}/out-publish-normalized"

## 02. variables

In [4]:
filename_infos = COMBINE_harmonizer.FILENAME_INFOS

## 03. Functions

### 03-3. process filename info

In [5]:
def _process_filename_info(filename_info):
    # 1. read df
    filename = filename_info['name']
    full_filename = os.sep.join([out_dir, filename])
    df =  pd.read_csv(full_filename, dtype='O')
    df = df.fillna('')
    valid_columns = list(filter(lambda column: not column.endswith('.orig'), df.columns))
    df = df[valid_columns]

    # 2. prefix.
    prefix = COMBINE_harmonizer.flatten_filename_prefix(filename)
    print(f'[INFO] _process_filename_info: filename: {filename} prefix: {prefix}')

    if COMBINE_harmonizer.FLATTEN_INDEX not in df:
        # 4. with prefix
        df_with_prefix = COMBINE_harmonizer.flatten_columns_with_prefix(df.copy(), prefix)

        return df, df_with_prefix

    # 3. flatten.
    df_flatten = COMBINE_harmonizer.flatten(df)

    # 4. with prefix.
    df_flatten_with_prefix = COMBINE_harmonizer.flatten_columns_with_prefix(df_flatten, prefix)

    return df, df_flatten_with_prefix

## 04. Process

In [6]:
out_filename = os.sep.join([out_dir, 'zz-merged.xlsx'])
excel = pd.ExcelWriter(out_filename, engine='xlsxwriter')

df_merge = None
for idx, filename_info in enumerate(filename_infos):
    if not filename_info.get('is_merge', True):
        continue

    # 1. names
    filename = filename_info['name']
    prefix = COMBINE_harmonizer.flatten_filename_prefix(filename)
    sheet_name = COMBINE_harmonizer.flatten_sheet_name(filename)
    print(f"({idx}/{len(filename_infos)}) filename: {filename_info['name']} name: {prefix} sheet_name: {sheet_name}")

    # 2. process file info.
    # df for sheet in excel.
    # df_flatten for merging as the flattened csv.
    df, df_flatten = _process_filename_info(filename_info)

    # 3. to excel.
    print(f'({idx}/{len(filename_infos)}) to df.excel')
    df.to_excel(excel, sheet_name=sheet_name)
    print(f'({idx}/{len(filename_infos)}) after df.excel')

    # 4. to merge.
    print(f'({idx}/{len(filename_infos)}) to merge')
    if df_merge is None:
        df_merge = df_flatten
    else:
        df_merge = df_merge.merge(df_flatten, on=COMBINE_harmonizer.FLATTEN_MERGE_COLUMNS, how='outer')

    # 5. check that unique-id is still unique.
    df_groupby = df_merge.groupby(COMBINE_harmonizer.FLATTEN_MERGE_COLUMNS).agg(count=('_study', 'count'))
    is_invalid = df_groupby['count'] > 1
    print(f'({idx}/{len(filename_infos)}) after merge: groupby: {len(df_groupby)} is_invalid: {is_invalid.sum()}')

# save to csv.
out_filename = os.sep.join([out_dir, 'zz-merged-flatten.csv'])
df_merge.to_csv(out_filename, index=False)

# merge to excel.
print(f'merged columns: {len(df_merge.columns)}')
# df_merge = df_merge.transpose()
df_merge.to_excel(excel, sheet_name='flatten')

# close excel.
print('to excel.close')
excel.close()
print('done')

(2/72) filename: 01-02-screening.csv name: 01-02 sheet_name: 01-02-screening
[INFO] _process_filename_info: filename: 01-02-screening.csv prefix: 01-02
(2/72) to df.excel
(2/72) after df.excel
(2/72) to merge
(2/72) after merge: groupby: 532 is_invalid: 0
(3/72) filename: 01-03-maternal-demographics.csv name: 01-03 sheet_name: 01-03-maternal-demographics
[INFO] _process_filename_info: filename: 01-03-maternal-demographics.csv prefix: 01-03
(3/72) to df.excel
(3/72) after df.excel
(3/72) to merge


(3/72) after merge: groupby: 532 is_invalid: 0
(4/72) filename: 01-04-pregnancy-history.csv name: 01-04 sheet_name: 01-04-pregnancy-history
[INFO] _process_filename_info: filename: 01-04-pregnancy-history.csv prefix: 01-04
(4/72) to df.excel
(4/72) after df.excel
(4/72) to merge
(4/72) after merge: groupby: 532 is_invalid: 0
(5/72) filename: 01-05-labor-delivery.csv name: 01-05 sheet_name: 01-05-labor-delivery
[INFO] _process_filename_info: filename: 01-05-labor-delivery.csv prefix: 01-05
(5/72) to df.excel
(5/72) after df.excel
(5/72) to merge
(5/72) after merge: groupby: 532 is_invalid: 0
(6/72) filename: 01-06-birth.csv name: 01-06 sheet_name: 01-06-birth
[INFO] _process_filename_info: filename: 01-06-birth.csv prefix: 01-06
(6/72) to df.excel


(6/72) after df.excel
(6/72) to merge
(6/72) after merge: groupby: 532 is_invalid: 0
(7/72) filename: 01-07-pre-temperature.csv name: 01-07 sheet_name: 01-07-pre-temperature
[INFO] _process_filename_info: filename: 01-07-pre-temperature.csv prefix: 01-07
(7/72) to df.excel
(7/72) after df.excel
(7/72) to merge
(7/72) after merge: groupby: 532 is_invalid: 0
(8/72) filename: 01-08-pre-cardiovascular.csv name: 01-08 sheet_name: 01-08-pre-cardiovascular
[INFO] _process_filename_info: filename: 01-08-pre-cardiovascular.csv prefix: 01-08
(8/72) to df.excel
(8/72) after df.excel
(8/72) to merge
(8/72) after merge: groupby: 532 is_invalid: 0
(9/72) filename: 01-09-pre-infection.csv name: 01-09 sheet_name: 01-09-pre-infection
[INFO] _process_filename_info: filename: 01-09-pre-infection.csv prefix: 01-09
(9/72) to df.excel
(9/72) after df.excel
(9/72) to merge
(9/72) after merge: groupby: 532 is_invalid: 0
(10/72) filename: 01-10-pre-other-med.csv name: 01-10 sheet_name: 01-10-pre-other-med


[INFO] _process_filename_info: filename: 01-10-pre-other-med.csv prefix: 01-10
(10/72) to df.excel
(10/72) after df.excel
(10/72) to merge
(10/72) after merge: groupby: 532 is_invalid: 0
(11/72) filename: 01-11-pre-imaging.csv name: 01-11 sheet_name: 01-11-pre-imaging
[INFO] _process_filename_info: filename: 01-11-pre-imaging.csv prefix: 01-11
(11/72) to df.excel
(11/72) after df.excel
(11/72) to merge
(11/72) after merge: groupby: 532 is_invalid: 0
(12/72) filename: 01-12-neuro-exam.csv name: 01-12 sheet_name: 01-12-neuro-exam
[INFO] _process_filename_info: filename: 01-12-neuro-exam.csv prefix: 01-12
(12/72) to df.excel
(12/72) after df.excel
(12/72) to merge


(12/72) after merge: groupby: 532 is_invalid: 0
(13/72) filename: 02-01-temperature.csv name: 02-01 sheet_name: 02-01-temperature
[INFO] _process_filename_info: filename: 02-01-temperature.csv prefix: 02-01
_flatten: to pivot_table: df: 24439


...

(68/72) after df.excel
(68/72) to merge
(68/72) after merge: groupby: 532 is_invalid: 0
(69/72) filename: 31-05-disability-level-death.csv name: 31-05 sheet_name: 31-05-disability-level-death
[INFO] _process_filename_info: filename: 31-05-disability-level-death.csv prefix: 31-05
(69/72) to df.excel
(69/72) after df.excel
(69/72) to merge
(69/72) after merge: groupby: 532 is_invalid: 0
(70/72) filename: 31-06-emergency-csection.csv name: 31-06 sheet_name: 31-06-emergency-csection
[INFO] _process_filename_info: filename: 31-06-emergency-csection.csv prefix: 31-06
(70/72) to df.excel
(70/72) after df.excel
(70/72) to merge
(70/72) after merge: groupby: 532 is_invalid: 0
(71/72) filename: 31-07-length-of-stay.csv name: 31-07 sheet_name: 31-07-length-of-stay


[INFO] _process_filename_info: filename: 31-07-length-of-stay.csv prefix: 31-07
(71/72) to df.excel
(71/72) after df.excel
(71/72) to merge
(71/72) after merge: groupby: 532 is_invalid: 0


merged columns: 6131


to excel.close


done
